In [ ]:
iterations = np.arange(10000)
for i, _ in enumerate(iterations):
    print(f"Iteration {i+1}/{len(iterations)}")

# Extract features and labels
    X_llm = llm_recommendations_final[financial_metric_cols].apply(pd.to_numeric, errors='coerce')
    y_llm = llm_recommendations_final["action"]

    # Train-test split
    X_train_llm, X_test_llm, y_train_llm, y_test_llm = train_test_split(
        X_llm, y_llm, test_size=0.2, random_state=i, stratify=y_llm 
    )

    # Fit and Predict
    rf_llm = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_llm.fit(X_train_llm, y_train_llm)
    y_pred_llm = rf_llm.predict(X_test_llm)

    # Add predictions to dataframe
    llm_recommendations_final.loc[X_test_llm.index, "pred_llm"] = y_pred_llm

    # Extract and split
    X_analyst = analyst_recommendations_final[financial_metric_cols].apply(pd.to_numeric, errors='coerce')
    y_analyst = analyst_recommendations_final["action"]

    X_train_analyst, X_test_analyst, y_train_analyst, y_test_analyst = train_test_split(
        X_analyst, y_analyst, test_size=0.2, random_state=i, stratify=y_analyst
    )

    # Fit and Predict
    rf_analyst = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_analyst.fit(X_train_analyst, y_train_analyst)
    y_pred_analyst = rf_analyst.predict(X_test_analyst)

    # Align to DF
    analyst_recommendations_final.loc[X_test_analyst.index, "pred_analyst"] = y_pred_analyst

    # Merge LLM and analyst recommendations with financial metrics
    merged_df = llm_recommendations_final.merge(
        analyst_recommendations_final, 
        on = ["cik", "date"],
        suffixes=('_llm', '_analyst'),
        how = 'left'
    )

    # Subset for non entry llm and analyst rating
    merged_df = merged_df[~merged_df["pred_llm"].isna() & ~merged_df["pred_analyst"].isna()]

    # LLM buy combinations
    try:
        llm_buy_analyst_buy_sample = merged_df[(merged_df["action_llm"] == "buy") & (merged_df["pred_llm"] == "buy") &
                                            (merged_df["action_analyst"] == "buy") & (merged_df["pred_analyst"] == "buy")].sample(n=1, random_state=42)
        llm_buy_analyst_hold_sample = merged_df[(merged_df["action_llm"] == "buy") & (merged_df["pred_llm"] == "buy") &
                                                (merged_df["action_analyst"] == "hold") & (merged_df["pred_analyst"] == "hold")].sample(n=1, random_state=42)
        llm_buy_analyst_sell_sample = merged_df[(merged_df["action_llm"] == "buy") & (merged_df["pred_llm"] == "buy") &
                                                (merged_df["action_analyst"] == "sell") & (merged_df["pred_analyst"] == "sell")].sample(n=1, random_state=42)

        # LLM hold combinations
        llm_hold_analyst_buy_sample = merged_df[(merged_df["action_llm"] == "hold") & (merged_df["pred_llm"] == "hold") &
                                                (merged_df["action_analyst"] == "buy") & (merged_df["pred_analyst"] == "buy")].sample(n=1, random_state=42)
        llm_hold_analyst_hold_sample = merged_df[(merged_df["action_llm"] == "hold") & (merged_df["pred_llm"] == "hold") &
                                                (merged_df["action_analyst"] == "hold") & (merged_df["pred_analyst"] == "hold")].sample(n=1, random_state=42)
        llm_hold_analyst_sell_sample = merged_df[(merged_df["action_llm"] == "hold") & (merged_df["pred_llm"] == "hold") &
                                                (merged_df["action_analyst"] == "sell") & (merged_df["pred_analyst"] == "sell")].sample(n=1, random_state=42)

        # LLM sell combinations
        llm_sell_analyst_buy_sample = merged_df[(merged_df["action_llm"] == "sell") & (merged_df["pred_llm"] == "sell") &
                                                (merged_df["action_analyst"] == "buy") & (merged_df["pred_analyst"] == "buy")].sample(n=1, random_state=42)
        llm_sell_analyst_hold_sample = merged_df[(merged_df["action_llm"] == "sell") & (merged_df["pred_llm"] == "sell") &
                                                (merged_df["action_analyst"] == "hold") & (merged_df["pred_analyst"] == "hold")].sample(n=1, random_state=42)
        llm_sell_analyst_sell_sample = merged_df[(merged_df["action_llm"] == "sell") & (merged_df["pred_llm"] == "sell") &
                                                (merged_df["action_analyst"] == "sell") & (merged_df["pred_analyst"] == "sell")].sample(n=1, random_state=42)
        print(f"Success! All 9 combinations found at seed i={i}")
        break 
    except ValueError as e:
        print(f"Sampling error in iteration {i+1}: {e}")
        continue

# Combine into one df
different_recs = pd.concat([
    llm_buy_analyst_buy_sample,
    llm_buy_analyst_hold_sample,
    llm_buy_analyst_sell_sample,
    llm_hold_analyst_buy_sample,
    llm_hold_analyst_hold_sample,
    llm_hold_analyst_sell_sample,
    llm_sell_analyst_buy_sample,
    llm_sell_analyst_hold_sample,
    llm_sell_analyst_sell_sample
]).reset_index(drop=True)
different_recs.sort_values(by=["cik", "date"])